In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))

In [2]:
from src.modules.pdf_interpreter import *
from src.modules.helper_fns import *

In [3]:
overall_df = generate_fin_df()

Reading PDFs:   0%|          | 0/18 [00:00<?, ?it/s]

APR 2022_eStatement.pdf
August 2022 e-Statement .pdf
December 2021 e-Statement .pdf
December 2022 e-Statement .pdf
February 2022 e-Statement .pdf
February 2023 e-Statement .pdf
January 2022 e-Statement .pdf
January 2023 e-Statement .pdf
July 2022 e-Statement .pdf
June 2022 e-Statement .pdf
March 2022 e-Statement.pdf
May 2022 e-Statement .pdf
November 2021 e-Statement .pdf
November 2022 e-Statement .pdf
October 2021 e-Statement .pdf
October 2022 e-Statement .pdf
September 2021 e-Statement .pdf
September 2022 e-Statement .pdf


Reading PDFs:   0%|          | 0/19 [00:00<?, ?it/s]

April 2022 e-Statement.pdf
August 2022 e-Statement.pdf
December 2021 e-Statement.pdf
December 2022 e-Statement.pdf
February 2022 e-Statement.pdf
February 2023 e-Statement.pdf
January 2022 e-Statement.pdf
January 2023 e-Statement.pdf
July 2022 e-Statement.pdf
June 2022 e-Statement.pdf
March 2022 e-Statement.pdf
March 2023 e-Statement.pdf
May 2022 e-Statement.pdf
November 2021 e-Statement.pdf
November 2022 e-Statement.pdf
October 2021 e-Statement.pdf
October 2022 e-Statement.pdf
September 2021 e-Statement.pdf
September 2022 e-Statement.pdf


In [4]:
overall_df

,Reference #,Transaction Date,Post Date,Details,Amount,Transaction Type,Year,Account,DateTime
0,005,Aug 10,Aug 10,MB-CREDIT CARD/LOC PAY. FROM -,40.91,*****04*1181 MR JA,2022,Credit,2022-08-10
1,006,Aug 11,Aug 12,AMZN Mktp CA*BM2420FT3,207.91,WWW.AMAZON.CA ON,2022,Credit,2022-08-11
2,007,Aug 12,Aug 15,A&W 4965 NEPEANON,7.10,For al,2022,Credit,2022-08-12
3,008,Aug 12,Aug 15,OCT-CONFEDERATION-PARLIAM OTTAWAON,20.00,most,2022,Credit,2022-08-12
4,009,Aug 13,Aug 15,Subway 4618Ottawa ON,14.10,point,2022,Credit,2022-08-13
...,...,...,...,...,...,...,...,...,...
892,NaN,Apr 8,NaN,Credit Card,123.41,MB-Transfer to,2022,Chequing,2022-04-08
893,NaN,Apr 11,NaN,Cra (Rev) Tax Amount Owing,200.00,TS Bill payment,2022,Chequing,2022-04-11
894,NaN,Apr 14,NaN,Opos Uber Canada Toronto ONCA,23.95,Point of sale purchase,2022,Chequing,2022-04-14
895,NaN,Apr 14,NaN,Opos Uber Canada Toronto ONCA,1.05,Point of sale purchase,2022,Chequing,2022-04-14
